In [3]:
!pip install -U dspy-ai openai python-dotenv pandas

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s  0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.3 MB/s  0:00:00
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 13.9 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 12.9 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from dotenv import load_dotenv
import dspy

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

if not OPENAI_API_KEY:
    raise RuntimeError("Missing OPENAI_API_KEY in environment/.env")

# Use DSPy's built-in OpenAI wrapper
lm = dspy.LM(model=OPENAI_MODEL, api_key=OPENAI_API_KEY, temperature=0.2)
dspy.settings.configure(lm=lm)

In [14]:
import pandas as pd

CSV_PATH = "insurance_letters_structured.csv"  # <-- your CSV
df = pd.read_csv(CSV_PATH).fillna("")

required = ["raw_text","letter_type","company_name","insured_name","policy_number","claim_number"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"CSV missing required columns: {missing}")

df.head(2)


,filename,company_name,letter_type,insured_name,policy_number,claim_number,raw_text
0,insurance_letter_0001.pdf,Acme Insurance Co.,Request for Additional Information,Sophia Garcia,P-2681365,C-5934663,Acme Insurance Co.\n123 Liberty Avenue\nNewark...
1,insurance_letter_0002.pdf,Liberty Shield Insurance,Denial Letter,Aarav Anderson,P-4955706,C-1439566,Liberty Shield Insurance\n123 Liberty Avenue\n...


In [15]:
class LetterToInstructions(dspy.Signature):
    raw_text: str = dspy.InputField(desc="Full insurance letter text")
    letter_type: str = dspy.InputField(desc="Coverage Decision / Denial Letter / Request for Additional Information")
    company_name: str = dspy.InputField(desc="Insurance company name")

    instructions_list: str = dspy.OutputField(
        desc="Step-by-step structured instructions to write a letter of this type, including required sections and compliance language."
    )

class InstructionSynthesizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.p = dspy.Predict(LetterToInstructions)

    def forward(self, raw_text: str, letter_type: str, company_name: str):
        return self.p(raw_text=raw_text, letter_type=letter_type, company_name=company_name)

In [16]:
class InstructionsToLetter(dspy.Signature):
    instructions_list: str = dspy.InputField(desc="Step-by-step instructions")
    letter_type: str = dspy.InputField(desc="Letter type")
    company_name: str = dspy.InputField(desc="Company")

    insured_name: str = dspy.InputField(desc="Insured name")
    policy_number: str = dspy.InputField(desc="Policy number like P-1234567")
    claim_number: str = dspy.InputField(desc="Claim number like C-1234567")

    drafted_letter: str = dspy.OutputField(desc="Generated insurance letter")

class LetterGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.p = dspy.Predict(InstructionsToLetter)

    def forward(self, **kwargs):
        return self.p(**kwargs)

In [17]:
import re
from difflib import SequenceMatcher

def has_phrase(text, phrase):
    return phrase.lower() in text.lower()

def similarity(a: str, b: str) -> float:
    # quick, cheap similarity (0..1)
    return SequenceMatcher(None, a[:4000], b[:4000]).ratio()

def eval_generated_letter(original: str, generated: str, company: str, policy: str, claim: str, letter_type: str) -> float:
    if not generated.strip():
        return 0.0

    score = 0.0

    # 1) Must contain key identifiers
    if company and has_phrase(generated, company): score += 0.15
    if policy and has_phrase(generated, policy): score += 0.15
    if claim and has_phrase(generated, claim): score += 0.15

    # 2) Basic structure signals
    if has_phrase(generated, "Subject:"): score += 0.08
    if has_phrase(generated, "Dear"): score += 0.08
    if has_phrase(generated, "Sincerely"): score += 0.08

    # 3) Compliance language
    compliance_hits = sum([
        has_phrase(generated, "Compliance"),
        has_phrase(generated, "Regulatory"),
        has_phrase(generated, "appeal"),
        has_phrase(generated, "policy"),
        has_phrase(generated, "laws") or has_phrase(generated, "regulations"),
    ])
    score += min(0.25, 0.05 * compliance_hits)

    # 4) Letter-type specific cues (light)
    lt = letter_type.lower()
    if "denial" in lt and (has_phrase(generated, "regret") or has_phrase(generated, "cannot be extended")):
        score += 0.06
    if "coverage" in lt and (has_phrase(generated, "approved") or has_phrase(generated, "coverage applies")):
        score += 0.06
    if "additional information" in lt and (has_phrase(generated, "additional") or has_phrase(generated, "documentation")):
        score += 0.06

    # 5) Rough similarity to original (keep small weight)
    score += 0.10 * similarity(original, generated)

    return max(0.0, min(1.0, score))

In [18]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def to_examples(frame: pd.DataFrame):
    exs = []
    for _, r in frame.iterrows():
        ex = dspy.Example(
            raw_text=r["raw_text"],
            letter_type=r["letter_type"],
            company_name=r["company_name"],
            insured_name=r["insured_name"],
            policy_number=r["policy_number"],
            claim_number=r["claim_number"],
            # for evaluation we keep the original letter as "target"
            original_letter=r["raw_text"],
        ).with_inputs("raw_text","letter_type","company_name","insured_name","policy_number","claim_number")
        exs.append(ex)
    return exs

trainset = to_examples(train_df)
valset = to_examples(val_df)

len(trainset), len(valset)

(400, 100)

In [22]:
synth = InstructionSynthesizer()
gen = LetterGenerator()

class Pipeline(dspy.Module):
    def __init__(self, synth, gen):
        super().__init__()
        self.synth = synth
        self.gen = gen

    def forward(self, raw_text, letter_type, company_name, insured_name, policy_number, claim_number):
        instr = self.synth(raw_text=raw_text, letter_type=letter_type, company_name=company_name)
        out = self.gen(
            instructions_list=instr.instructions_list,
            letter_type=letter_type,
            company_name=company_name,
            insured_name=insured_name,
            policy_number=policy_number,
            claim_number=claim_number
        )
        # return both for inspection
        return dspy.Prediction(
            instructions_list=instr.instructions_list,
            drafted_letter=out.drafted_letter
        )

pipe_student = Pipeline(synth, gen)

def pipeline_metric(example, pred, trace=None) -> float:
    return eval_generated_letter(
        original=example.original_letter,
        generated=getattr(pred, "drafted_letter", ""),
        company=example.company_name,
        policy=example.policy_number,
        claim=example.claim_number,
        letter_type=example.letter_type,
    )


In [23]:
for i in range(3):
    ex = valset[i]
    pred = pipe_student(**ex.inputs())
    s = pipeline_metric(ex, pred)
    print("\n--- Example", i, "score =", round(s, 3), "---")
    print("INSTRUCTIONS (first 400 chars):\n", pred.instructions_list[:400])
    print("\nLETTER (first 800 chars):\n", pred.drafted_letter[:800])


--- Example 0 score = 1.0 ---
INSTRUCTIONS (first 400 chars):
 1. **Header Section**: 
   - Include the insurance company name and address at the top.
   - Add the subject line indicating the type of letter (e.g., "Denial Letter").

2. **Salutation**:
   - Address the recipient by name (e.g., "Dear [Recipient's Name],").

3. **Claim Information**:
   - Clearly state the insurance policy number and claim number for reference.

4. **Claim Review Summary**:
   -

LETTER (first 800 chars):
 Cascade Assurance  
123 Insurance Ave  
City, State, Zip Code  

**Subject: Denial Letter**

Dear Michael Miller,

We are writing to inform you about the status of your claim associated with your insurance policy.

**Claim Information:**  
Policy Number: P-4927734  
Claim Number: C-1747681  

After a thorough review of your claim, we have considered all supporting documentation submitted. 

**Claim Review Summary:**  
Our review process included an evaluation of the circumstances surrounding your claim

In [24]:
mipro = dspy.MIPROv2(
    metric=pipeline_metric,
    auto="light",
    max_bootstrapped_demos=2,
    max_labeled_demos=2,
    verbose=True
)

# keep val small so it completes faster
val_small = valset[:20]

pipe_optimized = mipro.compile(
    pipe_student,
    trainset=trainset,
    valset=val_small,
    minibatch=False
)

print("✅ Optimization complete")

2025/12/28 07:46:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 20

2025/12/28 07:46:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/28 07:46:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/28 07:46:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 2/400 [00:00<00:26, 15.06it/s]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/6


  0%|          | 2/400 [00:33<1:50:22, 16.64s/it]


...


Predictor 0
i: You are a professional insurance letter writer. Given the fields `raw_text`, `letter_type`, `company_name`, produce a detailed list of step-by-step instructions, `instructions_list`, outlining how to create a response letter for the specified type, ensuring to include necessary sections and compliance language that adheres to insurance communication standards.
p: Instructions List:
Predictor 1
i: Using the information provided in the `instructions_list`, create a formally structured insurance letter that reflects the `letter_type` (such as "Coverage Decision" or "Denial Letter"), including the `company_name`, the `insured_name`, the `policy_number`, and the `claim_number`. Ensure the letter adheres to the prescribed format and contains all required content and compliance language.
p: Drafted Letter:


Average Metric: 20.00 / 20 (100.0%): 100%|██████████| 20/20 [00:19<00:00,  1.04it/s]

2025/12/28 07:58:48 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 20 (100.0%)
2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 100.0 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 3', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 4'].
2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [97.2, 100.0, 96.95, 95.11, 95.45, 96.67, 95.11, 93.77, 94.23, 98.33, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 94.47, 93.4, 100.0, 100.0]
2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 100.0
2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 21 / 20 =====
2025/12/28 07:58:48 INFO dspy.teleprompt.mipro_optimizer_v2: Evaluating the following candidate program...




Predictor 0
i: You are a skilled insurance correspondence specialist. Given the fields `raw_text`, `letter_type`, and `company_name`, produce a comprehensive and structured list of step-by-step instructions that will guide you in drafting a clear, legally compliant insurance letter. Ensure that your instructions encompass all necessary sections, include appropriate compliance language, and facilitate effective communication with the recipient.
p: Instructions List:
Predictor 1
i: Utilizing the provided `instructions_list`, create a fully drafted insurance letter by incorporating the specified `letter_type`, `company_name`, `insured_name`, `policy_number`, and `claim_number`. Ensure that the letter is well-structured, compliant with industry standards, and contains all necessary details that facilitate clear communication between the insurance provider and the client.
p: Drafted Letter:


Average Metric: 20.00 / 20 (100.0%): 100%|██████████| 20/20 [00:42<00:00,  2.11s/it]

2025/12/28 07:59:30 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 20 (100.0%)
2025/12/28 07:59:30 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 100.0 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 1', 'Predictor 1: Few-Shot Set 1'].
2025/12/28 07:59:30 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [97.2, 100.0, 96.95, 95.11, 95.45, 96.67, 95.11, 93.77, 94.23, 98.33, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 94.47, 93.4, 100.0, 100.0, 100.0]
2025/12/28 07:59:30 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 100.0
2025/12/28 07:59:30 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/12/28 07:59:30 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 100.0!



✅ Optimization complete


In [26]:
OUT_PATH = "extracted_instructions_from_letters.txt"

with open(OUT_PATH, "w", encoding="utf-8") as f:
    f.write("Extracted Instruction Sets (Predictor 0)\n")
    f.write("=" * 80 + "\n\n")

    for idx, ex in enumerate(valset[:50]):  # change slice as needed
        instr = pipe_optimized.synth(
            raw_text=ex.raw_text,
            letter_type=ex.letter_type,
            company_name=ex.company_name,
        ).instructions_list

        f.write(f"[Example {idx}]\n")
        f.write(f"Letter Type: {ex.letter_type}\n")
        f.write(f"Company: {ex.company_name}\n\n")
        f.write(instr.strip())
        f.write("\n\n" + "-" * 80 + "\n\n")

print(f"✅ Saved instruction outputs to {OUT_PATH}")

✅ Saved instruction outputs to extracted_instructions_from_letters.txt


In [28]:
from datetime import datetime

class CanonicalInstructionSignature(dspy.Signature):
    """
    Merge and deduplicate multiple instruction sets into a single
    comprehensive, professional instruction guide for generating
    insurance letters (all types).
    """
    instructions_corpus: str = dspy.InputField(
        desc="Multiple instruction sets extracted from many insurance letters."
    )
    canonical_instructions: str = dspy.OutputField(
        desc=(
            "One detailed, deduplicated instruction guide that covers ALL letter types "
            "(Coverage Decision, Denial Letter, Request for Additional Information). "
            "Must include: required sections, formatting, placeholders, compliance language, "
            "and type-specific variations."
        )
    )

class CanonicalInstructionBuilder(dspy.Module):
    def __init__(self):
        super().__init__()
        self.p = dspy.Predict(CanonicalInstructionSignature)

    def forward(self, instructions_corpus: str):
        return self.p(instructions_corpus=instructions_corpus)


In [30]:

def collect_predictor0_instructions(
    pipe,
    dataset,
    max_per_type=10,
):
    """
    Collect Predictor-0 instruction outputs stratified by letter_type.
    Prevents running all 400 examples.
    """
    collected = []
    seen = {}

    for ex in dataset:
        lt = ex.letter_type
        seen.setdefault(lt, 0)

        if seen[lt] >= max_per_type:
            continue

        out = pipe.synth(
            raw_text=ex.raw_text,
            letter_type=ex.letter_type,
            company_name=ex.company_name,
        )

        instr = (out.instructions_list or "").strip()
        if instr:
            collected.append(instr)
            seen[lt] += 1

        # stop early if all types satisfied
        if all(v >= max_per_type for v in seen.values()):
            break

    return collected


# Collect from full trainset + the same val_small you used in MIPRO
instr_train = collect_predictor0_instructions(
    pipe_optimized,
    trainset,
    max_per_type=12   # ~36 total if 3 letter types
)

instr_val = collect_predictor0_instructions(
    pipe_optimized,
    val_small,
    max_per_type=5    # ~15 total
)

all_instr = instr_train + instr_val
print(f"Collected {len(all_instr)} instruction sets from Predictor 0")

Collected 49 instruction sets from Predictor 0


In [31]:

# Build corpus text (the model will dedupe/merge)
instructions_corpus = "\n\n---\n\n".join(all_instr)

# Create canonical instructions (one-time call)
canonical_builder = CanonicalInstructionBuilder()
canonical_instructions = canonical_builder(
    instructions_corpus=instructions_corpus
).canonical_instructions

# Save to TXT
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_PATH = f"canonical_insurance_letter_instructions_{ts}.txt"

with open(OUT_PATH, "w", encoding="utf-8") as f:
    f.write("CANONICAL INSTRUCTIONS FOR INSURANCE LETTER GENERATION\n")
    f.write("=" * 90 + "\n\n")
    f.write(canonical_instructions.strip())
    f.write("\n")

print(f"✅ Canonical instruction set saved to: {OUT_PATH}")

✅ Canonical instruction set saved to: canonical_insurance_letter_instructions_20251228_091201.txt
